In [ ]:
import pystac_client
import folium
from odc import stac as odc_stac
from ascat.read_native.ragged_array_ts import CellFileCollection
import xarray as xr
from pathlib import Path
import numpy as np
from pyproj import Transformer
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
import yaml
import holoviews as hv
import dask

dask.config.set(**{"array.slicing.split_large_chunks": True})

hv.extension("bokeh")
import hvplot.xarray

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    # h121_ds = h121_ds.set_index(time="time")

In [ ]:
paths = yaml.safe_load(Path("../paths.yml").read_text())
root: Path = Path(paths["linux"]).expanduser()
cell_source: Path = root / "datasets/scat_ard/ascat_ssm_cdr_12.5km_h121"
assert cell_source.exists()

### Loading Sentinel - 1 Sigma Naught Data from EODC STAC Catalogue

In [ ]:
eodc_catalog = pystac_client.Client.open("https://stac.eodc.eu/api/v1")

colllection_id = "SENTINEL1_SIG0_20M"

collection = eodc_catalog.get_collection(colllection_id)

Setting time and area of interest.

In [ ]:
time_range = "2021-06-01/2021-09-30"

latmin, latmax = 47, 48  # South to North
lonmin, lonmax = 15, 16.5  # West to East

bounding_box = [lonmin, latmin, lonmax, latmax]

Loading the metadata with STAC search engine.

In [ ]:
search = eodc_catalog.search(
    collections=colllection_id,
    bbox=bounding_box,
    datetime=time_range,
    # max_items=1  # number of max items to load
)
items_eodc = search.item_collection()
print(f"On EODC we found {len(items_eodc)} items for the given search query")

Let's plot thumbnail of the loaded items for this area and those dates.

In [ ]:
map = folium.Map(
    location=[(latmin + latmax) / 2, (lonmin + lonmax) / 2],
    zoom_start=7,
    zoom_control=False,
    scrollWheelZoom=False,
    dragging=False,
)

folium.Rectangle(
    bounds=[[latmin, lonmin], [latmax, lonmax]],
    color="blue",
    fill=True,
    fill_opacity=0.1,
    weight=2,
    popup="Area of Interest",
).add_to(map)

for item in items_eodc:
    # url leading to display of an item, can also be used as hyperlink
    image_url = item.assets["thumbnail"].href
    bounds = item.bbox
    folium.raster_layers.ImageOverlay(
        image=image_url,
        bounds=[[bounds[1], bounds[0]], [bounds[3], bounds[2]]],
    ).add_to(map)

folium.LayerControl().add_to(map)

map

Now, let's load the data with `odc_stac`.

In [ ]:
crs = "EPSG:27704"  # Coordinate Reference System: EQUI7 Grid of Europe
res = 20  # meter
chunks = {"time": 1, "latitude": 1000, "longitude": 1000}
sig0_dc = odc_stac.load(
    items_eodc,
    crs=crs,
    resolution=res,
    bbox=bounding_box,
    chunks=chunks,
    resampling="bilinear",
)

sig0_dc

In [ ]:
item = items_eodc[-1]
scale = item.assets["VV"].extra_fields.get("raster:bands")[0]["scale"]
nodata = item.assets["VV"].extra_fields.get("raster:bands")[0]["nodata"]
sig0_dc = sig0_dc.where(sig0_dc != nodata) / scale

sig0_dc = sig0_dc.dropna(dim="time", how="all", subset=["VV"])  # .compute()
sig0_dc

In [ ]:
n = 20

ds_vv = (
    sig0_dc["VV"]
    .isel(x=slice(None, None, 10), y=slice(None, None, 10))
    .isel(time=slice(0, n))
)

ncols = 5
nrows = (n + ncols - 1) // ncols

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(4 * ncols, 4 * nrows))

axes = axes.flatten()

for i in range(n):
    vv_t = ds_vv.isel(time=i)
    ax = axes[i]
    im = vv_t.plot(ax=ax, robust=True, cmap="viridis", add_colorbar=False)
    ax.set_title(str(vv_t["time"].values)[:19])  # Shorten timestamp
    ax.set_xlabel("x [m]")
    ax.set_ylabel("y [m]")

fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label="VV [dB]")

plt.suptitle("Sentinel-1 VV (Downsampled) - First 20 Time Steps", fontsize=16)
plt.tight_layout(rect=[0, 0, 0.9, 0.95])
plt.show()


EODC has items that do not overlap and are about 25 seconds apart, as that is Sentinel-1 radar acquisition time.

In [ ]:
vv_t0 = sig0_dc.VV.isel(time=4)
vv_t1 = sig0_dc.VV.isel(time=5)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))

vv_t0.plot(ax=axes[0], robust=True, cmap="viridis")
axes[0].set_title(f"Sentinel-1 VV - Time 0\n{str(vv_t0.time.values)}")

vv_t1.plot(ax=axes[1], robust=True, cmap="viridis")
axes[1].set_title(f"Sentinel-1 VV - Time 1\n{str(vv_t1.time.values)}")

plt.tight_layout()
plt.show()

So to avoid having double the amount of data per time instance, lets merge EODC items per hour. We can also merge per minute, or even day, since for this area we have data every ~4 days, so that is the rough temporal resolution. But let's keep it precision in the hourly range.

In [ ]:
# sig0_dc2 = sig0_dc.resample(time="d").first()

In [ ]:
hourly_time = sig0_dc.time.dt.floor("h")
sig0_dc = sig0_dc.assign_coords(hourly_time=("time", hourly_time.data))
sig0_dc = sig0_dc.groupby("hourly_time").mean(dim="time")
sig0_dc = sig0_dc.rename({"hourly_time": "time"})

sig0_dc

In [ ]:
n = 20

ds_vv = (
    sig0_dc["VV"]
    .isel(x=slice(None, None, 10), y=slice(None, None, 10))
    .isel(time=slice(0, n))
)

ncols = 5
nrows = (n + ncols - 1) // ncols

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(4 * ncols, 4 * nrows))

axes = axes.flatten()

for i in range(n):
    vv_t = ds_vv.isel(time=i)
    ax = axes[i]
    im = vv_t.plot(ax=ax, robust=True, cmap="viridis", add_colorbar=False)
    ax.set_title(str(vv_t["time"].values)[:19])  # Shorten timestamp
    ax.set_xlabel("x [m]")
    ax.set_ylabel("y [m]")


fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label="VV [dB]")

plt.suptitle("Sentinel-1 VV (Downsampled) - First 20 Time Steps", fontsize=16)
plt.tight_layout(rect=[0, 0, 0.9, 0.95])
plt.show()


We see that we have half of the time instances now. So let's plot first merged item (radar image) to see were images (that were acquired on the same day and in same hour) on the previous plot merged.

In [ ]:
sig0_dc.isel(time=0).VV.plot(size=8, robust=True)

### Loading EUMETSAT H SAF ASCAT H121 data from local TU Wien directory

Let's extract `gpis` - grid point indices from read data.

In [ ]:
h121_reader = CellFileCollection.from_product_id(cell_source, "H121_V1.0")
gpis, lons, lats, cells = h121_reader.grid.get_grid_points()

Now, we will use specified bounding box that was used to filter out EODC's Sentinel-1 data to filter out grid point indices that are inside this area.

In [ ]:
indices = np.where(
    (lats >= latmin) & (lats <= latmax) & (lons >= lonmin) & (lons <= lonmax)
)[0]

print("There are", len(indices), "grid point indices inside specified bounding box")

Let's make an array of those indices that will be used to load individual time series of each point into an `xarray` dataarrays. Those dataarrays will be combined into `xarray` dataset, which will store time series of each individual grid point.

In [ ]:
selected_coords = list(zip(lons[indices], lats[indices]))
selected_gpis = gpis[indices]

In [ ]:
datasets = []

for i, id in enumerate(selected_gpis):
    print(f"Loading location_id: {id} ({i + 1}/{len(selected_gpis)})")

    ds = h121_reader.read(location_id=id)

    ds = ds.swap_dims({"obs": "time"})

    ds = ds.expand_dims(location_id=[id])
    datasets.append(ds)  # loop takes around 15 seconds

ascat_ds = xr.concat(datasets, dim="location_id")  # this will also take a few seconds

In [ ]:
ascat_ds = ascat_ds.sel(time=slice(*time_range.split("/")))
ascat_ds

In [ ]:
print(ascat_ds.time.values[0:50])

Finally, let's plot the results to see how the grid points look like and what is their spatial resolution.

In [ ]:
lats = ascat_ds["lat"].values
lons = ascat_ds["lon"].values
ids = ascat_ds["location_id"].values

center = [lats.mean(), lons.mean()]
map = folium.Map(location=center, zoom_start=8)

folium.Rectangle(
    bounds=[[latmin, lonmin], [latmax, lonmax]],
    color="blue",
    fill=True,
    fill_opacity=0.1,
    weight=2,
    popup="Area of Interest",
).add_to(map)

for lat, lon, loc_id in zip(lats, lons, ids):
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        popup=f"ID: {loc_id}",
        color="blue",
        fill=True,
        fill_color="blue",
    ).add_to(map)

map


### Fusing Sentinel-1 and ASCAT H SAF datasets

#### 1) Regridding Sentinel-1 dataset to the ASCAT grid

Let's firstly check the projections of the two datasets, keeping in mind that ASCAT is in WGS84 (EPSG:4326).

In [ ]:
sig0_crs = sig0_dc.rio.crs
print("Coordinate reference system of Sentinel-1 dataset:", sig0_crs)

The reprojection has to be done in pairs of latitudes and longitudes, since we are not working with raster, but rather a grid of points.

In [ ]:
# Set up transformer from EPSG:4326 (lat/lon) to EPSG:27704 (Equi7 Grid)
transformer = Transformer.from_crs("EPSG:4326", "EPSG:27704", always_xy=True)

ascat_lons = ascat_ds["lon"].values
ascat_lats = ascat_ds["lat"].values

ascat_xs, ascat_ys = transformer.transform(ascat_lons, ascat_lats)

# Assign new projected coordinates
ascat_reproj_ds = ascat_ds.copy()
ascat_reproj_ds = ascat_reproj_ds.assign_coords(
    {"x": ("location_id", ascat_xs), "y": ("location_id", ascat_ys)}
)

ascat_reproj_ds

Now, when we have same coordinate reference system for both datasets, let's sample Sentinel-1 to the the grid of ASCAT data- in other words, ASCAT dataset will be master here and its grid will be the reference.

In [ ]:
ascat_points = ascat_reproj_ds[["x", "y"]].to_dataframe().dropna().reset_index()

s1_regridded_ds = sig0_dc.sel(
    x=xr.DataArray(ascat_points["x"].values, dims="location_id"),
    y=xr.DataArray(ascat_points["y"].values, dims="location_id"),
    method="nearest",
)

s1_regridded_ds = s1_regridded_ds.assign_coords(
    location_id=("location_id", ascat_points["location_id"].values),
    x=("location_id", ascat_points["x"].values),
    y=("location_id", ascat_points["y"].values),
)

s1_regridded_ds = s1_regridded_ds.transpose("time", "location_id")

s1_regridded_ds = s1_regridded_ds.compute()  # takes a minute
s1_regridded_ds

Be aware that simple
```
sig0_dc.sel(
    x=xr.DataArray(ascat_points["x"].values, dims="location_id"),
    y=xr.DataArray(ascat_points["y"].values, dims="location_id"),
    method="nearest",
)
```
would find a nearest pixel that has data for at a time instance. If closest pixel (let's say 1 meter away) has no value on specific date, but a pixel 400 km away has, it would add it's backscatter value to the time series, which is not so physical. So we would have combined time series of a pixel 1 m away and also 400 000 m away.

In [ ]:
ascat_point = ascat_reproj_ds.isel(location_id=0)
target_x = ascat_point["x"].item()
target_y = ascat_point["y"].item()


In [ ]:
s1_x = sig0_dc["x"].values
s1_y = sig0_dc["y"].values

x_idx = np.abs(s1_x - target_x).argmin()
y_idx = np.abs(s1_y - target_y).argmin()

In [ ]:
s1_y - target_y

In [ ]:
vv_series = sig0_dc["VV"].isel(x=x_idx, y=y_idx)
vh_series = sig0_dc["VH"].isel(x=x_idx, y=y_idx)

In [ ]:
vv_series.sel(time=slice("2021-06-01", "2021-06-15")).plot()


Since we do not have raster data, but rather gridded vector data, it is the best to plot it as scatter plot, if we are interested in how geospatially gridded Sentinel-1 data looks like.

In [ ]:
time_index = 0
time_sel = s1_regridded_ds["time"].values[time_index]

plt.figure(figsize=(8, 6))

# Plotting original Sentinel-1 raster data
sig0_dc.isel(time=time_index).VV[::10, ::10].plot(
    robust=True, cmap="gray", add_colorbar=False, alpha=0.5
)

# Plotting gridded Sentinel-1 data
scatterplot = plt.scatter(
    ascat_reproj_ds["x"].values,
    ascat_reproj_ds["y"].values,
    c=s1_regridded_ds.VV.sel(time=time_sel).values,
    cmap="viridis",
    s=50,
    edgecolor="k",
)

plt.colorbar(scatterplot, label="Sentinel VV (sampled)")
plt.title(f"Sentinel VV at ASCAT Grid Points\nTime: {str(time_sel)}")
plt.xlabel("x [m]")
plt.ylabel("y [m]")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
loc_id = 10

ascat_sm = ascat_ds.isel(location_id=loc_id).dropna(dim="time").surface_soil_moisture
s1_vv = s1_regridded_ds.isel(location_id=loc_id).dropna(dim="time").VV

fig, ax1 = plt.subplots(figsize=(12, 5))

# ASCAT on primary y-axis
color1 = "tab:blue"
ax1.set_xlabel("Time")
ax1.set_ylabel("ASCAT Surface Soil Moisture", color=color1)
ax1.plot(ascat_sm.time, ascat_sm, color=color1, label="ASCAT SM")
ax1.tick_params(axis="y", labelcolor=color1)

# Sentinel-1 on secondary y-axis
ax2 = ax1.twinx()
color2 = "tab:red"
ax2.set_ylabel("Sentinel-1 VV", color=color2)
ax2.plot(s1_vv.time, s1_vv, color=color2, label="S1 VV")
ax2.tick_params(axis="y", labelcolor=color2)

plt.title(f"Location ID: {ascat_ds.location_id.values[loc_id]}")
fig.tight_layout()
plt.show()


In [ ]:
s1_regridded_ds.isel(location_id=loc_id).sel(
    time=slice("2021-06-01", "2021-06-15")
).dropna(dim="time").VV.plot()

In [ ]:
sig0_dc.isel(x=400, y=400).sel(time=slice("2021-06-01", "2021-06-15")).dropna(
    dim="time"
).VV.plot()

### 2) Temporal Matching

One can notice that ASCAT data has considerably higher frequency then Sentinel-1. In fact, let's check it programmatically.

In [ ]:
ascat_series = ascat_ds.isel(location_id=loc_id).dropna("time")
s1_series = s1_regridded_ds.isel(location_id=loc_id).dropna("time")

ascat_counts = ascat_series.resample(time="1D").count()
s1_counts = s1_series.resample(time="1D").count()

ascat_obs_per_day = ascat_counts.surface_soil_moisture.mean().item()

s1_times = s1_series["time"].values
s1_time_diffs = (s1_times[1:] - s1_times[:-1]) / np.timedelta64(1, "D")
s1_avg_interval = np.mean(s1_time_diffs)

print(f"ASCAT avg observations per day: {ascat_obs_per_day:.2f}")
print(f"Sentinel-1 avg revisit time: {s1_avg_interval:.2f} days")

plt.figure(figsize=(10, 5))
ascat_counts.surface_soil_moisture.plot(label="ASCAT", linewidth=1)
s1_counts.VV.plot(label="Sentinel-1", linewidth=2)
plt.title(f"Temporal Sampling per Day at Location Index: {loc_id}")
plt.ylabel("Observations per Day")
plt.xlabel("Date")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
ascat_times = ascat_ds["time"]
s1_expanded = s1_regridded_ds.sel(time=ascat_times, method="nearest")

s1_expanded["time"] = ascat_times

In [ ]:
s1_expanded.VV